In [7]:
from super_gradients.common.object_names import Models
from super_gradients.training import models
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.conversion.conversion_enums import ExportQuantizationMode
from super_gradients.conversion import ExportTargetBackend

#model = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")


model = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")
"""model = models.get(
    Models.YOLO_NAS_L,
    num_classes=3,
    checkpoint_path=f"average_model.pth"
)"""

export_result = model.export(
    "yolonas_padel_l.onnx",
    confidence_threshold = 0.1,
    nms_threshold = 0.5,
    num_pre_nms_predictions = 100,
    max_predictions_per_image = 50,
    output_predictions_format = DetectionOutputFormatMode.FLAT_FORMAT,
    #engine=ExportTargetBackend.TENSORRT
)


[2024-01-31 00:54:27] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-01-31 00:54:27] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l
[2024-01-31 00:54:28] INFO - checkpoint_utils.py - Successfully loaded model weights from average_model.pth checkpoint.
c:\Users\USER\.conda\envs\py39\lib\site-packages\torch\onnx\symbolic_opset9.py:5408: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


In [ ]:
export_result

In [ ]:
!trtexec --onnx=yolo_nas_l_trt.onnx --uint8 --avgRuns=100 --duration=15

In [47]:
import cv2
import numpy as np
from super_gradients.training.utils.media.image import load_image
import onnxruntime
import time

image = load_image("https://deci-pretrained-models.s3.amazonaws.com/sample_images/beatles-abbeyroad.jpg")
image = cv2.resize(image, (640, 640))
image_bchw = np.transpose(np.expand_dims(image, 0), (0, 3, 1, 2))


In [48]:
session = onnxruntime.InferenceSession("yolo_nas_s_top_5.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
inputs = [o.name for o in session.get_inputs()]
outputs = [o.name for o in session.get_outputs()]
start = time.perf_counter()
result = session.run(outputs, {inputs[0]: image_bchw})
end = time.perf_counter()
print(f"Time: {end-start} ms")

print(result)

In [49]:
flat_predictions = result
for (_, x_min, y_min, x_max, y_max, confidence, class_id) in flat_predictions[0]:
    class_id = int(class_id)
    print(f"Detected object with class_id={class_id}, confidence={confidence}, x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}")

In [50]:
import matplotlib.pyplot as plt
from super_gradients.training.datasets.datasets_conf import COCO_DETECTION_CLASSES_LIST
from super_gradients.training.utils.detection_utils import DetectionVisualization

def show_predictions_from_flat_format(image, predictions):
    [flat_predictions] = predictions

    image = image.copy()
    class_names = COCO_DETECTION_CLASSES_LIST
    color_mapping = DetectionVisualization._generate_color_mapping(len(class_names))

    for (sample_index, x1, y1, x2, y2, class_score, class_index) in flat_predictions[flat_predictions[:, 0] == 0]:
        class_index = int(class_index)
        image = DetectionVisualization.draw_box_title(
                    image_np=image,
                    x1=int(x1),
                    y1=int(y1),
                    x2=int(x2),
                    y2=int(y2),
                    class_id=class_index,
                    class_names=class_names,
                    color_mapping=color_mapping,
                    box_thickness=2,
                    pred_conf=class_score,
                )

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.tight_layout()
    plt.show()

In [ ]:
show_predictions_from_flat_format(image, result)